# 単眼深度推定モデルを学習する
このノートではいよいよ単眼深度推定モデルの学習を行う。
ここでは簡単のために、最もシンプルなLossとネットワークをもちいて大枠の学習の方法を見ていく。

## ハイパーパラメタ
まずはじめにハイパーパラメタを設定しておく

In [ ]:
frame_inds = [0, -1]  # 隣接フレームの番号
epochs = 20
lr = 0.0001
batch_size = 24

## ネットワーク
学習するネットワークは現在のフレームの深度を推定するものと、フレーム間の姿勢の変化を推定するものの２つである。<br>
ここではそれぞれdepth netとpose netと呼ぶ。
depth netは深度マップを推定するネットワークであり、pose netは回転（X軸,Y軸,Z軸）と並進（X,Y,Z）の合計6個の数値を推定するネットワークである。

### depth net
ここではネットワークの詳細に関心はないため、`segmentation_models_pytorch` をつかいU-Netを定義する。

In [ ]:
import segmentation_models_pytorch as smp
import torch

depth_net = smp.Unet('resnet18', in_channels=3, encoder_weights="imagenet", classes=1, activation='sigmoid')

# 入出力確認
depth_net(torch.zeros(1, 3, 224, 224))

### pose net

pose netに関してもネットワークの詳細には関心がないため、torchvisionの学習済みモデルを持ってきて、部分的にレイヤーを差し替える。<br>
pose netの入力は現在フレームと隣接フレームの２枚であり、出力はそれらのフレーム間での姿勢の変化量であるため、入力チャンネルは合計で6チャンネルとなる。

In [ ]:
from torchvision.models import resnet18
from torch.nn import Linear, Conv2d

pose_net = resnet18(pretrained=True)
pose_net.conv1 = Conv2d(in_channels=3*2, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
pose_net.fc = Linear(in_features=512, out_features=6)

# 入出力確認
pose_net(torch.zeros(1, 3*2, 224, 224)).shape

ネットワークの定義は以上である。

## Loss関数
ここではPhotometric lossを定義する。<br>

In [ ]:
import sys
sys.path.append("../../src")

In [ ]:
import torch.nn.functional as F
from camera import PinholeCamera
from functools import lru_cache

class PhotometricLoss(torch.nn.Module):
    def __init__(self, frame_inds):
        super().__init__()
        self.frame_inds = frame_inds
        self.l1_loss = torch.nn.L1Loss()

    def forward(self, y, y_pred):
        image_target = y["rgb_0"]
        depth = y_pred[f"depth_0"]
        intrinsic = y["intrinsic_0"]
        loss = 0
        for idx in self.frame_inds:
            # target 同士は比較しない
            if idx == 0:
                continue
            image_source = y[f"rgb_{idx}"]
            extrinsic_src2tgt = y_pred[f"extrinsic_{idx}"]
            image_warped = self.warp(image_source, depth, intrinsic, extrinsic_src2tgt)
            loss += self.l1_loss(image_warped, image_target)
        return loss, image_warped

    def warp(self, image_source, depth, intrinsic, extrinsic_src2tgt):
        image_coords = self.create_image_coords(depth.shape)
        image_coords = image_coords.to(depth.device)
        world_coords = PinholeCamera.image2world(image_coords, intrinsic, extrinsic_src2tgt, depth, batch=True)
        # skip world2camera() because the coord is already camera coords here.
        world_coords = world_coords[..., :3] # remove 4th dim        
        image_coords = PinholeCamera.camera2image(world_coords, intrinsic, batch=True)        
        # normalize the coord to be in [-1, 1] for grid sampling
        image_coords[..., 0] = image_coords[..., 0] / image_coords.shape[2] * 2 - 1
        image_coords[..., 1] = image_coords[..., 1] / image_coords.shape[1] * 2 - 1
        grid = image_coords
        image_warped = F.grid_sample(image_source, grid)
        return image_warped
                
    @lru_cache(None)
    def create_image_coords(self, map_shape):
        # 以前と同様にmeshgridで画像座標を生成する
        xi = torch.arange(0, map_shape[2], 1)
        yi = torch.arange(0, map_shape[1], 1)
        coord_x, coord_y = torch.meshgrid(xi, yi, indexing="xy")
        image_coords = torch.stack([coord_x, coord_y], axis=-1)
        image_coords = image_coords.float()
        # batch
        image_coords = image_coords.unsqueeze(0).repeat(map_shape[0], 1, 1, 1)
        return image_coords

In [ ]:
criterion = PhotometricLoss(frame_inds=frame_inds)

## Pose Utils
pose netが推定した姿勢の変化量（6変数）を4x4の行列に変換するユーティリティ関数を定義する。
と思ったが、torchgeometryで提供されている関数が便利だったため、それを使うことにする。

In [ ]:
import torchgeometry as tgm

tgm.rtvec_to_pose(torch.rand(3, 6)).shape


## training loop

In [ ]:
from dataset import VKITTI2

train_dataset = VKITTI2(root_dir="../../data")

In [ ]:
from torch.utils.data import DataLoader, Dataset
import cv2


class Transform(Dataset):
    def __init__(self, dataset):        
        super().__init__()
        self.dataset = dataset
        self.crop = (45, 43, 1197, 331) # (x0, y0, x1, y1)
        self.scale = 1.0 / 3.0

    def __getitem__(self, idx):
        data = self.dataset[idx]
        for key in data.keys():
            if key.startswith("rgb_") or key.startswith("depth_"):
                image = data[key]
                image = image[self.crop[1]:self.crop[3], self.crop[0]:self.crop[2]]
                orig_shape = image.shape
                dest_size = (int(orig_shape[1] * self.scale), int(orig_shape[0] * self.scale))
                image = cv2.resize(image, dest_size) 
                data[key] = torch.tensor(image).float()
                if key.startswith("rgb_"):
                    data[key] = data[key].permute(2, 0, 1) # (B, H, W, C) -> (B, C, H, W)
                    data[key] /= 255.0 # normalize
            elif key.startswith("intrinsic_"):
                intrinsic = data[key]
                # apply crop offset and scale
                intrinsic[0, 2] = intrinsic[0, 2] - self.crop[0]
                intrinsic[1, 2] = intrinsic[1, 2] - self.crop[1]
                intrinsic[:2, :] *= self.scale
                data[key] = intrinsic
            
            data[key] = torch.tensor(data[key]).float()

        return data

    def __len__(self):
        return len(self.dataset)


train_dataloader = DataLoader(Transform(train_dataset), batch_size=batch_size, shuffle=True, drop_last=True)

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

device="cuda:0"

depth_net.train().to(device)
pose_net.train().to(device)

optimizer = Adam([
    {"params": depth_net.parameters()},
    {"params": pose_net.parameters()}],
    lr=lr)


for i in range(epochs):
    with tqdm(train_dataloader) as pbar:
        for batch in pbar:
            # transport batch to device
            batch = {k:v.to(device) for k, v in batch.items()}

            optimizer.zero_grad()

            # predict depth
            inv_depth = depth_net(batch["rgb_0"])
            inv_depth = inv_depth.squeeze(1)
            inv_depth = torch.clip(inv_depth, min=0.001, max=100)
            depth = 1 / inv_depth

            # precict pose
            image_concat = torch.cat([batch["rgb_0"], batch["rgb_-1"]], axis=1)
            pose = pose_net(image_concat)
            rtmat = tgm.rtvec_to_pose(pose)

            # compute loss
            y = {k:v for k, v in batch.items() if k.startswith("rgb_") or k.startswith("intrinsic_")}
            y_pred = {
                "depth_0": depth,
                "extrinsic_-1": rtmat,
            }
            loss, image_warped = criterion(y, y_pred)
            loss.backward()
            optimizer.step()

            # status update
            pbar.set_description(f"[Epoch {i}] loss: {loss:0.3f}, depth mean {depth.mean():0.3f}")
